## Without the package

In [1]:
import numpy as np

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
valid_size = 0.2
batch_size = 32

transform_training = transforms.Compose(
    [transforms.RandomRotation(15),   
     transforms.RandomResizedCrop(224),
     transforms.RandomHorizontalFlip(),
     transforms.ColorJitter(saturation=0.2, contrast=0.2, hue=0.2, brightness=0.2),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)]
)
transform_testing = transforms.Compose(
    [transforms.Resize((224,224)),
    transforms.ToTensor(), 
    transforms.Normalize(mean, std)]
)

train_data = datasets.ImageFolder("./raw_images_clean/train", transform=transform_training)
valid_data = datasets.ImageFolder("./raw_images_clean/valid", transform=transform_training)
test_data = datasets.ImageFolder("./raw_images_clean/test", transform=transform_testing)

# And now we create the loders
train_loader = DataLoader(train_data, batch_size=batch_size)
valid_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)


loaders_scratch = {'train': train_loader, 'valid': valid_loader, 'test': test_loader}

print(len(train_loader), len(valid_loader), len(test_loader))

395 395 16


In [2]:
# useful variable that tells us whether we should use the GPU
import torch
use_cuda = torch.cuda.is_available()

In [3]:
import torch.nn as nn
from torch import optim
criterion_scratch = nn.CrossEntropyLoss()

def get_optimizer_scratch(model):
    return optim.Adam(model.parameters(), lr=0.0003)

In [30]:
import torch.nn as nn
import torch.nn.functional as F

number_classes = len(train_data.classes)

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv_1 = nn.Conv2d(3, 16, 3, stride=1, padding=1) 
        self.conv_2 = nn.Conv2d(16, 32, 3, stride=1, padding=1) 
        self.conv_3 = nn.Conv2d(32, 64, 3, stride=1, padding=1) 
        self.conv_4 = nn.Conv2d(64, 128, 3, stride=1, padding=1) 
        self.conv_5 = nn.Conv2d(128, 256, 3, stride=1, padding=1) 

        
        # And also define a max pooling layer
        self.pool = nn.MaxPool2d(2,2)        
        
        # And linear models
        self.fc1 = nn.Linear(7*7*256, 1024) # 7 as (224/(2*2*2*2*2) and 256 as it is the depth

        # Other model that i tested
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, number_classes)

        
        # Will define a dropout of 0.3
        self.dropout = nn.Dropout(0.2)
        
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv_1(x)))
        x = self.pool(F.relu(self.conv_2(x)))
        x = self.pool(F.relu(self.conv_3(x)))
        x = self.pool(F.relu(self.conv_4(x)))
        x = self.pool(F.relu(self.conv_5(x)))

        
        # Flatten the input
        x = x.view(-1, 7*7*256)
        
        # Add the dropout
        x = self.dropout(x)
        
        # Add the linear layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

model_scratch

Net(
  (conv_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [32]:
number_classes

100

In [26]:
class_values = {
    'first': 100,
    'second': 100
}

In [36]:
import torch.nn as nn
import torch.nn.functional as F


# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        self.conv_1 = nn.Conv2d(3, 16, 3, stride=1, padding=1) 
        self.conv_2 = nn.Conv2d(16, 32, 3, stride=1, padding=1) 
        self.conv_3 = nn.Conv2d(32, 64, 3, stride=1, padding=1) 
        self.conv_4 = nn.Conv2d(64, 128, 3, stride=1, padding=1) 
        self.conv_5 = nn.Conv2d(128, 256, 3, stride=1, padding=1) 

        
        # And also define a max pooling layer
        self.pool = nn.MaxPool2d(2,2)        
        
        # And linear models
        self.fc1 = nn.Linear(7*7*256, 1024) # 7 as (224/(2*2*2*2*2) and 256 as it is the depth

        # Other model that i tested
        for class_name, class_length in class_values.items():
            setattr(self, f'fc2_{class_name}', nn.Linear(1024, 512))
            setattr(self, f'fc3_{class_name}', nn.Linear(512, 256))
            setattr(self, f'fc4_{class_name}', nn.Linear(256, class_length))
        
        # Will define a dropout of 0.3
        self.dropout = nn.Dropout(0.2)
        
    
    def forward(self, x):
        ## Define forward behavior
        x = self.pool(F.relu(self.conv_1(x)))
        x = self.pool(F.relu(self.conv_2(x)))
        x = self.pool(F.relu(self.conv_3(x)))
        x = self.pool(F.relu(self.conv_4(x)))
        x = self.pool(F.relu(self.conv_5(x)))

        
        # Flatten the input
        x = x.view(-1, 7*7*256)
        
        # Add the dropout
        x = self.dropout(x)
        
        # Add the linear layers
        forward_values = {}
        for class_name in class_values.keys():
            new_x = self.dropout(F.relu(self.fc1(x)))
            new_x = self.dropout(getattr(self, f'fc2_{class_name}')(new_x))
            new_x = self.dropout(getattr(self, f'fc3_{class_name}')(new_x))
            new_x = getattr(self, f'fc4_{class_name}')(new_x)
            
            forward_values[class_name] = new_x
        
        # The class groups should be ordered, so those values always refer to the same classes
        return forward_values

# instantiate the CNN
model_scratch = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

model_scratch

Net(
  (conv_1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=1024, bias=True)
  (fc2_first): Linear(in_features=1024, out_features=512, bias=True)
  (fc3_first): Linear(in_features=512, out_features=256, bias=True)
  (fc4_first): Linear(in_features=256, out_features=100, bias=True)
  (fc2_second): Linear(in_features=1024, out_features=512, bias=True)
  (fc3_second): Linear(in_features=512, out_features=256, bias=True)
  (fc4_second): Linear(in_features=256, out_features=100, bias=True)
  (dropout): Dropout(p=0.2, inplace=Fals

In [34]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        # set the module to training mode
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            print(len(data))
            print(len(data[0][0]))
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()
            # Obtain the output from the model
            output = model(data)
            print(output)
            # Obtain loss
            loss = criterion(output, target)
            # Backward induction
            loss.backward()
            # Perform optimization step
            optimizer.step()            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))

        # set the model to evaluation mode
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            ## TODO: update average validation loss 
            output = model(data)
            # Obtain the loss
            loss = criterion(output, target)
            # Add this loss to the list (same as before but instead of train we use valid)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))
            

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
    
        if valid_loss < valid_loss_min:
            # Print an alert
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model..'.format(
                valid_loss_min,
                valid_loss))

            torch.save(model.state_dict(), save_path)
            
            # Update the new minimum
            valid_loss_min = valid_loss
        
    return model

In [37]:
def custom_weight_init(m):
    ## TODO: implement a weight initialization strategy
    classname = m.__class__.__name__
    
    # for every Linear layer in a model >
    if classname.find('Linear') != -1:
        # Get the number of the inputs
        n = m.in_features
        # Define which will be the y
        y = (1.0/np.sqrt(n))
        m.weight.data.normal_(0, y) 
        # Bias will be filled with 0
        m.bias.data.fill_(0)


#-#-# Do NOT modify the code below this line. #-#-#
    
model_scratch.apply(custom_weight_init)
model_scratch = train(
    20, 
    loaders_scratch, 
    model_scratch, 
    get_optimizer_scratch(model_scratch),
    criterion_scratch, 
    use_cuda, 
    'ignore.pt'
)

32
224
dict_values([tensor([[ 0.0086,  0.0019, -0.0150,  ...,  0.0110,  0.0545, -0.0010],
        [-0.0183, -0.0015, -0.0223,  ...,  0.0086,  0.0076, -0.0296],
        [-0.0327, -0.0335, -0.0487,  ...,  0.0316,  0.0016, -0.0094],
        ...,
        [-0.0180,  0.0075, -0.0163,  ...,  0.0351,  0.0477, -0.0118],
        [-0.0468, -0.0073, -0.0206,  ...,  0.0903,  0.0268, -0.0271],
        [-0.0460, -0.0276, -0.0260,  ...,  0.0534,  0.0180, -0.0045]],
       grad_fn=<AddmmBackward0>), tensor([[-0.0080,  0.0181, -0.0105,  ...,  0.0280, -0.0249, -0.0092],
        [ 0.0077, -0.0040, -0.0041,  ..., -0.0455,  0.0127,  0.0393],
        [ 0.0131, -0.0138, -0.0218,  ..., -0.0217, -0.0067,  0.0304],
        ...,
        [-0.0120, -0.0588, -0.0025,  ..., -0.0625, -0.0088,  0.0424],
        [ 0.0240, -0.0408, -0.0691,  ..., -0.0432, -0.0225,  0.0405],
        [-0.0141, -0.0127, -0.0546,  ..., -0.1041,  0.0604,  0.0646]],
       grad_fn=<AddmmBackward0>)])


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not dict_values

## With Auto Deep Learning

In [2]:
import os
import logging
import pandas as pd
from typing import List, Dict


def create_df_image_folder(
    columns: List[str], dtypes: Dict[str, str]
) -> pd.DataFrame:
    """
    Create the dataframe that will be the base for the image folder dataset.
    
    Parameters
    ----------
    columns : list
        The columns that will be used for the dataframe
    
    dtypes : dict
        Dictionary for the column name : column dtype for the dataframe
    
    Returns
    -------
    df : pd.DataFrame
        The DataFrame that will be used for organizing the Image Folder information
    """

    df = pd.DataFrame(columns=columns)

    df = df.astype(dtypes)

    return df


def image_folder_convertion(
    parent_folder_path: str,
    columns: List[str] = [
        'image_path',
        'class',
        'split_type'
    ],
    dtypes: Dict[str, str] = {
        'image_path': 'object',
        'class': 'category',
        'split_type': 'category'
    }
):
    """
    Function to make the conversion from the image folder structure to a dataframe that is wanted.
    
    Parameters
    ----------
    parent_folder_path : str
        The folder for which we have the ['train', 'test'] folders.

    columns : list
        The columns that will be used for the dataframe
    
    dtypes : dict
        Dictionary for the column name : column dtype for the dataframe
    
    Returns
    -------
    df : pd.DataFrame
        The DataFrame that will be used for organizing the Image Folder information
    """

    # Get the df we will use as the base one 
    df = create_df_image_folder(
        columns=columns,
        dtypes=dtypes
    )

    # We get the folders, which we expect to be ['train', 'test', 'valid']
    split_folders: List[str] = os.listdir(parent_folder_path)
    
    for split_folder in split_folders:
        # We get the path of .../train and .../test
        split_path: str = parent_folder_path + '/' + split_folder
    
        # List the values that we have inside of each folder, which would be the classes
        children_classes: List[str] = os.listdir(split_path)

        # For each class of each folder
        for child_class in children_classes:
            # Get the path of .../train/class
            images_path: str = split_path + '/' + child_class

            # List the images we have inside that folder
            images_child_class_path: List[str] = os.listdir(images_path) 

            for idx, file in enumerate(images_child_class_path):
                images_child_class_path[idx] = images_path + '/' + file
                
            class_list: List[str] = [child_class] * len(images_child_class_path)
            dtype_list: List[str] = [split_folder] * len(images_child_class_path)

            # TODO: This is hardtypping the columns, so not flexible (ImageFolder is not flexible neither)
            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        data = {
                            'class': class_list,
                            'split_type': dtype_list,
                            'image_path': images_child_class_path,
                    })
                ], axis=0
            )

            del images_path
            del images_child_class_path
            del class_list
            del dtype_list

    return df

In [110]:
df = image_folder_convertion('./raw_images_clean')

In [285]:
df['classification'] = df['class']
del df['class']

In [108]:
class Creator(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        transformation: transforms.Compose,
        class_groups = ['class']
    ):

        # TODO: Use _ for internal variables
        self.df = df     
        self.transformation = transformation
        self.class_groups = class_groups
        
        # TODO: Get index and class map

        # TODO: Make them as properties
        self.df_dummies = {}
        self.dict_mapping_idx_class = {}
        # self.__get_dummies()"""
        
        for class_group in self.class_groups:
            self.__get_dummies_df(class_group)
            self.__get_dummies_mapping(class_group)

    def __len__(self):
        return len(self.df)
    
    

    """def __get_dummies(
        self
    ):

        for class_group in self.class_groups:
            # Get the dummies & and mapping idx-class_value  for that class group
            self.__get_dummies_df(class_group)
            self.__get_dummies_mapping(class_group)
    """

    def __get_dummies_df(
        self,
        class_group: str
    ):

        dummy_df: pd.DataFrame = pd.get_dummies(
            pd.DataFrame(self.df.loc[:, class_group]), 
            columns=[class_group],
            prefix='',
            prefix_sep=''
        )

        self.df_dummies[class_group] = dummy_df

        return dummy_df

    
    def __get_dummies_mapping(
        self,
        class_group: str
    ):

        # For that class group, get which are the columns in the dummmies and create the mapping between idx and class name
        self.dict_mapping_idx_class[class_group] = {}

        for idx, class_value in enumerate(self.df_dummies[class_group].columns.values):
            self.dict_mapping_idx_class[class_group][str(idx)] = class_value
        
        return self.dict_mapping_idx_class

    @property
    def columns(self) -> List[str]:
        return self.df.columns.values.tolist()


    @columns.setter
    def columns(self, column_names: List[str]) -> List[str]:
        self.df.columns = column_names
        return self.df.columns.values.tolist()
    
    """@property
    def class_groups_list(self) -> List[List[str]]:
        return self.class_groups
    

    @property
    def class_group_num(self) -> int:
        return len(self.class_groups)"""


    @property
    def class_group_unique(
        self,
        column: str
    ):
    
        return self.df[column].unique().tolist()


    def __getitem__(
        self,
        idx
    ):

        """if torch.is_tensor(idx):
            idx = idx.tolist()"""

        # Get which is the image path
        img_path = self.df.iloc[idx, 0]
        image = Image.open(img_path)

        class_groups_index = {}
        for class_group in self.class_groups:            
            # Get positional index and convert into tensor
            class_groups_index[class_group] = torch.tensor(
                list(self.df_dummies[class_group].iloc[idx]).index(1),
            )
        
        # Return as many classes as class that we have, as well as the image
        sample = [image, class_groups_index]

        # If we have transformations, apply it
        if self.transformation:
            sample[0] = self.transformation(sample[0])

        return sample


In [103]:
creator.columns = ['img', 'classification', 'split']

In [104]:
creator.columns

['img', 'classification', 'split']

In [111]:
creator = Creator(df, transform_training)
creator_loader = DataLoader(creator, batch_size=16)
for idx, row in enumerate(creator_loader):
    print(row)

[tensor([[[[-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268],
          [-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268],
          [-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268],
          ...,
          [-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268],
          [-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268],
          [-1.8268, -1.8268, -1.8268,  ..., -1.8268, -1.8268, -1.8268]],

         [[-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381],
          ...,
          [-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381],
          [-1.7381, -1.7381, -1.7381,  ..., -1.7381, -1.7381, -1.7381]],

         [[-1.5081, -1.5081, -1.5081,  ..., -1.5081, -1.5081, -1.5081],
          [-1.5081, -1.5081, 

[tensor([[[[-1.9638, -1.9638, -1.9638,  ..., -1.5185, -1.5014, -1.5014],
          [-1.9638, -1.9638, -1.9638,  ..., -1.5185, -1.5014, -1.5014],
          [-1.9467, -1.9467, -1.9295,  ..., -1.4672, -1.5185, -1.5185],
          ...,
          [ 0.1768,  0.1426,  0.1254,  ..., -2.1179, -2.0837, -2.0837],
          [ 0.3138,  0.2967,  0.2796,  ..., -2.1008, -2.0837, -2.0837],
          [ 0.3994,  0.3309,  0.2967,  ..., -2.1008, -2.0837, -2.0837]],

         [[-0.6176, -0.6176, -0.6176,  ...,  0.4678,  0.4678,  0.4678],
          [-0.6176, -0.6176, -0.6176,  ...,  0.4678,  0.4678,  0.4678],
          [-0.6001, -0.6001, -0.5826,  ...,  0.4678,  0.4678,  0.4678],
          ...,
          [-0.4601, -0.4601, -0.4251,  ..., -2.0357, -2.0007, -2.0007],
          [-0.3025, -0.3375, -0.2850,  ..., -2.0182, -2.0007, -2.0007],
          [-0.2850, -0.2850, -0.2500,  ..., -2.0182, -2.0007, -2.0007]],

         [[-1.5604, -1.5604, -1.5604,  ..., -1.1944, -1.2119, -1.2119],
          [-1.5604, -1.5604, 

[tensor([[[[-1.4672, -1.4500, -1.3644,  ..., -0.8164, -0.3712, -0.2684],
          [-1.4672, -1.4329, -1.3644,  ..., -0.7822, -0.3198, -0.1828],
          [-1.4158, -1.3987, -1.3644,  ..., -0.7137, -0.1828, -0.0287],
          ...,
          [-0.9534, -0.9534, -0.9705,  ..., -0.5767, -0.5938, -0.5938],
          [-0.9534, -0.9705, -0.9877,  ..., -0.5767, -0.6109, -0.6109],
          [-0.9534, -0.9705, -0.9877,  ..., -0.5767, -0.6109, -0.6109]],

         [[-0.0224,  0.0126,  0.1176,  ..., -0.6527, -0.1800, -0.0399],
          [ 0.0126,  0.0301,  0.1352,  ..., -0.6176, -0.0924,  0.0301],
          [ 0.0826,  0.1176,  0.2227,  ..., -0.5301,  0.0476,  0.2052],
          ...,
          [-1.3004, -1.3004, -1.3179,  ..., -1.4755, -1.4930, -1.4930],
          [-1.3004, -1.3004, -1.3354,  ..., -1.5105, -1.4930, -1.4930],
          [-1.3004, -1.3004, -1.3354,  ..., -1.5105, -1.5455, -1.4930]],

         [[-0.3753, -0.3578, -0.2707,  ..., -0.3753,  0.0779,  0.1999],
          [-0.3578, -0.3404, 

[tensor([[[[-1.9638, -1.9638, -1.9638,  ...,  2.0777,  2.0092,  1.9407],
          [-1.9638, -1.9638, -1.9638,  ...,  2.0434,  1.9749,  1.8893],
          [ 1.9578,  1.9920,  2.0092,  ...,  2.0434,  1.9749,  1.9064],
          ...,
          [ 2.0605,  1.8893,  1.2385,  ...,  0.8789,  0.8789,  1.1529],
          [ 2.0263,  2.0434,  1.3584,  ...,  1.1529,  1.1872,  1.4783],
          [ 1.9920,  1.9235,  1.8722,  ...,  1.5468,  1.5468,  1.7180]],

         [[-1.8782, -1.8782, -1.8782,  ...,  2.2185,  2.0784,  1.9559],
          [-1.8782, -1.8782, -1.8782,  ...,  2.0609,  1.9209,  1.7808],
          [ 1.6232,  1.6232,  1.5882,  ...,  1.8859,  1.7108,  1.5707],
          ...,
          [ 1.0280,  0.8529,  0.1352,  ..., -0.0049, -0.0924,  0.1702],
          [ 1.0280,  1.0455,  0.3277,  ...,  0.3803,  0.3803,  0.7129],
          [ 0.9930,  0.9580,  0.9055,  ...,  0.9230,  0.9230,  1.1856]],

         [[-1.6476, -1.6476, -1.6476,  ...,  2.3437,  2.1868,  2.0648],
          [-1.6476, -1.6476, 

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 

KeyboardInterrupt: 

In [63]:
creator.dict_mapping_idx_class['class']['21']

AttributeError: 'Creator' object has no attribute 'dict_mapping_idx_class'

TypeError: 'int' object is not callable